#### This Notebook is for database search using peptide strings and experimental spectra as input to the network.

In [93]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import random as rand
import queue
import csv
from collections import OrderedDict
from IPython.display import clear_output
import csv
import re
from heapq import merge
from sklearn import preprocessing
from operator import itemgetter
import bisect
import pickle
import os
import gc
from configparser import ConfigParser
import ast
import sys
from pathlib import Path
import pickle
import shutil
import progressbar

from sklearn.datasets import make_circles, make_moons
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributed as dist
import torch.multiprocessing as mp
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from IPython.core.debugger import set_trace
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from IPython.display import display
from os import listdir
from os.path import isfile, join

from src.snaputils import simulatespectra as sim
from src.snaptrain import process
from src.snaputils import reader

In [101]:
# Temporary config func. Original one in the project.
class config:
    """Define constants"""
    AAMass = OrderedDict([('A', 71.037114), ('C', 103.009185), ('D', 115.026943), ('E', 129.042593),
                          ('F', 147.068414), ('G', 57.021464), ('H', 137.058912), ('I', 113.084064),
                          ('K', 128.094963), ('L', 113.084064), ('M', 131.040485), ('N', 114.042927),
                          ('P', 97.052764), ('Q', 128.058578), ('R', 156.101111), ('S', 87.032028),
                          ('T', 101.047679), ('V', 99.068414), ('W', 186.079313), ('Y', 163.0633),
                          ('p', 79.97), ('o', 15.99), ('h', 0.98), ('c', 57.02), ('a', 42.01),
                          ('r', -17.03), ('y', 43.01), ('d', -18.01), ('t', 26.02)])

    ModMass = {"Oxidation": 15.994915, "CAM": 57.02146, "Carbamidomethyl": 57.02146, "ICAT_light": 227.12,
               "ICAT_heavy": 236.12, "AB_old_ICATd0": 442.20, "AB_old_ICATd8": 450.20, "Acetyl": 42.0106,
               "Deamidation": 0.9840, "Pyro-cmC": -17.026549, "Pyro-glu": -17.026549, "Pyro_glu": -18.010565,
               "Amide": -0.984016, "Phospho": 79.9663, "Methyl": 14.0157, "Carbamyl": 43.00581}

    ModCHAR = OrderedDict([("15.99", "o"), ("0.98", "h"), ("57.02", "c"), ("42.01", "a"), ("-17.03", "r"),
                           ("79.97", "p"), ("43.01", "y"), ("-18.01", "d"), ("26.02", "t")])
    # ModCHAR = {"15.99": "o", "0.98": "h", "57.02": "c", "42.01": "a", "-17.03": "r", "79.97": "p"}
    Ignore = ["U", "X"]
    Mods = [#{"mod_char": "p", "aas": ["nt", "S", "T", "Y"]}
            {"mod_char": "o", "aas": ["M"]}
           ]
    H2O = 18.015
    NH3 = 17.031
    PROTON = 1.00727647
    DEFAULT_PARAM_PATH = os.path.join(os.getcwd(), 'config.ini')
    PARAM_PATH = None
    l_config = None


    def get_config(section='input', key=None):
        """Read the configuration parameters and return a dictionary."""

        # If file path is given use it otherwise use default.
        file_path = config.PARAM_PATH if config.PARAM_PATH else config.DEFAULT_PARAM_PATH

        # Read config and convert each value to appropriate type.
        # Only for the first time.
        if not config.l_config:
            config.l_config = dict()
            config_ = ConfigParser()
            assert isinstance(file_path, str)
            config_.read(file_path)
            for section_ in config_.sections():
                config.l_config[section_] = dict()
                for key_ in config_[section_]:
                    try:
                        config.l_config[section_][key_] = ast.literal_eval(config_[section_][key_])
                    except (ValueError, SyntaxError):
                        config.l_config[section_][key_] = config_[section_][key_]

        if section and section in config.l_config:
            if key and key in config.l_config[section]:
                return config.l_config[section][key]
            return config.l_config[section]
        return config.l_config

In [3]:
print(config.get_config(section='input', key='charge'))
print(config.get_config(section='input', key='use_mods'))
print(config.get_config(section='ml', key='batch_size'))
print(config.get_config(section='input', key='num_species'))

5
True
1024
9


In [95]:
class Net(nn.Module):
    def __init__(self, vocab_size, output_size=512, embedding_dim=512, hidden_lstm_dim=1024, lstm_layers=2):
        super(Net, self).__init__()

        self.spec_size = config.get_config(section='input', key='spec_size')
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.hidden_lstm_dim = hidden_lstm_dim
        self.embedding_dim = embedding_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_lstm_dim, self.lstm_layers,
                            # dropout=0.5, 
                            batch_first=True, bidirectional=True)
        # self.lstm = nn.DataParallel(self.lstm)
        
        self.linear1_1 = nn.Linear(self.spec_size, 512) # self.spec_size, 1024
        self.linear1_2 = nn.Linear(512, 256)            # 1024, 512
        #self.linear1_3 = nn.Linear(512, 256)

        self.linear2_1 = nn.Linear(self.hidden_lstm_dim * 2, 512) # 2048, 1024
        self.linear2_2 = nn.Linear(512, 256) # 1024, 512
        #self.linear2_3 = nn.Linear(256, 128)

        do = 0.5
        self.dropout1 = nn.Dropout(do)
        self.dropout2 = nn.Dropout(do)
        print("dropout: {}".format(do))
        #self.dropout3 = nn.Dropout(0.3)
        

    def forward(self, data, hidden, data_type=None):
        assert not data_type or data_type == "specs" or data_type == "peps"
        res = []
        if not data_type or data_type == "specs":
            specs = data[0]
            out = self.linear1_1(specs.view(-1, self.spec_size))
            out = F.relu(out)

            out = self.dropout2(out)
            out = self.linear1_2(out)
            out = F.relu(out)
            
            out_spec = F.normalize(out)
            res.append(out_spec)

        if not data_type or data_type == "peps":
            for peps in data[1:]:
                embeds = self.embedding(peps)
                lstm_out, hidden = self.lstm(embeds, hidden)
                lstm_out = lstm_out[:, -1, :]
                out = lstm_out.contiguous().view(-1, self.hidden_lstm_dim * 2)

                out = self.dropout1(out)
                out = self.linear2_1(out)
                out = F.relu(out)

                out = self.dropout1(out)
                out = self.linear2_2(out)
                out = F.relu(out)

                out_pep = F.normalize(out)
                res.append(out_pep)
        res.append(hidden)

        return res

    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers * 2, batch_size, self.hidden_lstm_dim).zero_(),
                      weight.new(self.lstm_layers * 2, batch_size, self.hidden_lstm_dim).zero_())
        return hidden
    
    def name(self):
        return "Net"

In [96]:
def create_out_dir(dir_path, exist_ok=True):
    out_path = Path(dir_path)
    if out_path.exists() and out_path.is_dir():
        if not exist_ok:
            shutil.rmtree(out_path)
            out_path.mkdir()
    else:
        out_path.mkdir()
        
    Path(join(out_path, 'spectra')).mkdir()
    Path(join(out_path, 'peptides')).mkdir()
    
def verify_in_dir(dir_path, ext, ignore_list=[]):
    in_path = Path(dir_path)
    assert in_path.exists() and in_path.is_dir()
    
    files = [join(dir_path, f) for f in listdir(dir_path) if
                 isfile(join(dir_path, f)) and not f.startswith('.') 
                 and f.split('.')[-1] == ext and f not in ignore_list]
    assert len(files) > 0
    return files

def isfloat(str_float):
    try:
        float(str_float)
        return True
    except ValueError: 
        return False

In [97]:
def mod_repl(match):
    lookup = str(round(float(match.group(0)), 2))
    return config.ModCHAR[lookup] if lookup in config.ModCHAR else ""

def preprocess_mgfs(mgf_dir, out_dir):
    
    mgf_files = verify_in_dir(mgf_dir, "mgf")
    create_out_dir(out_dir, exist_ok=False)
        
    print('reading {} files'.format(len(mgf_files)))
    
    spec_size = config.get_config(section='input', key='spec_size')
    charge = config.get_config(section='input', key='charge')
    use_mods = config.get_config(section='input', key='use_mods')
    num_species = config.get_config(section='input', key='num_species')
    seq_len = config.get_config(section='ml', key='pep_seq_len')
    
    ch = np.zeros(20)
    modified = 0
    unmodified = 0
    unique_pep_set = set()
    
    pep_dict = {}
    idx_spec_map = []
    pep_spec = []
    pep_idx = 0
    
    summ = np.zeros(spec_size)
    sq_sum = np.zeros(spec_size)
    N = 0
    
    tot_count = 0
    max_peaks = max_moz = 0
    for species_id, mgf_file in enumerate(mgf_files):
        print('Reading: {}'.format(mgf_file))
        
        f = open(mgf_file, "r")
        lines = f.readlines()
        f.close()
        
        count = lcount = 0
        
        pep_list = []
        dataset = []
        label = []
        
        mass_ign = 0
        pep_len_ign = 0
        dup_ign = 0

        print('len of file: ' + str(len(lines)))
        limit = 200000
        pep = []
        spec = []
        pep_set = set()
        is_name = is_mw = is_charge = False
        prev = 0
        i = 0
        while i < len(lines) and limit > 0:
            line = lines[i]
            i += 1

            if line.startswith('PEPMASS'):
                count += 1
                mass = float(re.findall(r"PEPMASS=([-+]?[0-9]*\.?[0-9]*)", line)[0])
                if round(mass)*10 < spec_size:
                    is_mw = True
                    # limit = limit - 1
                else:
                    is_name = is_mw = is_charge = False
                    mass_ign += 1
                    continue
            
            if is_mw and line.startswith('CHARGE'):
                l_charge = int(re.findall(r"CHARGE=([-+]?[0-9]*\.?[0-9]*)", line)[0])
                is_charge = True
                
            if is_mw and is_charge:
            
                ind = [] # setting the precision to one decimal point.
                val = []
                for ch_val in range(l_charge):
                    ind.append(ch_val)
                    val.append(1)

                while not isfloat(re.split(' |\t|=', lines[i])[0]):
                    i += 1
                    
                while 'END IONS' not in lines[i].upper():
                    if lines[i] == '\n':
                        i += 1
                        continue
                    mz_line = lines[i]
                    i += 1
                    mz_splits = re.split(' |\t', mz_line)
                    moz, intensity = float(mz_splits[0]), float(mz_splits[1])
                    if moz > max_moz:
                        max_moz = moz
                    if 0 < round(moz*10) < spec_size:
                        # spec[round(moz*10)] += round(intensity)
                        if ind[-1] == moz*10:
                            val[-1] += intensity
                        else:
                            ind.append(round(moz*10))
                            val.append(intensity)
                            
                ind = np.array(ind)
                val = np.array(val)
                val = (val - np.amin(val)) / (np.amax(val) - np.amin(val))
                for ch_val in range(l_charge):
                    val[ch_val] = 1
                assert len(ind) == len(val)
                spec = np.array([ind, val])
                
                summ[ind] += val
                sq_sum[ind] += val**2
                N += 1

                is_name = True

            if is_name and is_mw and is_charge:
                is_name = is_mw = is_charge = False

                """output the data to """
                spec_file_name = '{}-{}-{}.npy'.format(lcount, mass, l_charge)
                np.save(join(out_dir, 'spectra', spec_file_name), spec)

                lcount += 1
                tot_count += 1
                
                pep = 0
                spec = []
                new = int((i / len(lines)) * 100)
                if new >= prev + 10:
                    #clear_output(wait=True)
                    print('count: ' + str(lcount))
                    print(str(new) + '%')
                    prev = new

        #print('max peaks: ' + str(max_peaks))
        print('In current file, read {} out of {}'.format(lcount, count))
        print("Ignored: large mass: {}, pep len: {}, dup: {}".format(mass_ign, pep_len_ign, dup_ign))
        print('overall running count: ' + str(tot_count))
        print('max moz: ' + str(max_moz))
#         return pep_list, dataset, label
#         tmp_pep_list, tmp_dataset, tmp_labels = read_msp(msp_file, species_id, decoy)
#         pep_list.extend(tmp_dataset)
#         dataset.extend(tmp_dataset)
#         label.extend(tmp_labels)

    # save the map. this will be used to generate masks for hard positive/negative mining during training.
    # np.save(join(out_dir, "idx_spec_map.npy"), idx_spec_map)
    # with open(join(out_dir, 'pep_spec.pkl'), 'wb') as f:
    #     pickle.dump(pep_spec, f)
    
    print("Statistics:")
    print("Charge distribution:")
    print(ch)
    print("Modified:\t{}".format(modified))
    print("Unmodified:\t{}".format(unmodified))
    print("Unique Peptides:\t{}".format(len(unique_pep_set)))
    print("Sum: {}".format(summ))
    print("Sum-Squared: {}".format(sq_sum))
    print("N: {}".format(N))
    means = summ / N
    print("mean: {}".format(means))
    stds = np.sqrt((sq_sum / N) - means**2)
    stds[stds < 0.0000001] = float("inf")
    print("std: {}".format(stds))
    np.save(join(out_dir, 'means.npy'), means)
    np.save(join(out_dir, 'stds.npy'), stds)

# return spectra, masses, charges

In [98]:
mgf_dir = "/disk/raptor-2/mtari008/data/deepsnap/pxd000612/mgfs-small"
# mgf_dir = "/disk/raptor-2/mtari008/data/deepsnap/preprocessed-human-hcd-tryp-best/mgfs"
# out_dir = "/disk/raptor-2/mtari008/data/deepsnap/preprocessed-human-hcd-tryp-best/pts"
out_dir = "/disk/raptor-2/mtari008/data/deepsnap/preprocessed/"
preprocess_mgfs(mgf_dir, out_dir)

reading 1 files
Reading: /disk/raptor-2/mtari008/data/deepsnap/preprocessed-human-hcd-tryp-best/mgfs/mgf-for-crux.mgf
len of file: 5367984
count: 3968
10%
count: 8058
20%
count: 12085
30%
count: 16184
40%
count: 20068
50%
count: 24132
60%
count: 28349
70%
count: 32265
80%
count: 36201
90%
In current file, read 40514 out of 40514
Ignored: large mass: 0, pep len: 0, dup: 0
overall running count: 40514
max moz: 4883.5847
Statistics:
Charge distribution:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Modified:	0
Unmodified:	0
Unique Peptides:	0
Sum: [40514. 40514. 16583. ...     0.     0.     0.]
Sum-Squared: [40514. 40514. 16583. ...     0.     0.     0.]
N: 40514
mean: [1.        1.        0.4093153 ... 0.        0.        0.       ]
std: [       inf        inf 0.49170752 ...        inf        inf        inf]


In [99]:
def find_occurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

In [102]:
def apply_mod(peps, mod):
    result_peps = set()
    pep_seq_len = config.get_config(section="ml", key="pep_seq_len")
    for pep in peps:
        if len(pep) >= pep_seq_len:
            continue
        for mod_aa in mod["aas"]:
            if mod_aa == "nt" and not pep[1].islower():
                result_peps.update([pep[0] + mod["mod_char"] + pep[1:]])
            elif mod_aa == "ct" and not pep[-1].islower():
                result_peps.update([pep + mod["mod_char"]])
            else:
                aa_indices = find_occurrences(pep, mod_aa)
                for index in aa_indices:
                    if index == len(pep) - 1 or not pep[index+1].islower():
                        result_peps.update([pep[:index+1] + mod["mod_char"] + (pep[index+1:] if index < len(pep) else "")])
        
    return result_peps

def add_mods(pep, mods, num_mods):
    mod_peps = set([pep])
    result_peps = set([pep])
    for i in range(num_mods):
        temp_mod_peps = set()
        for mod in mods:
            temp_mod_peps.update(apply_mod(mod_peps, mod))
        mod_peps.update(set(temp_mod_peps))
        result_peps.update(mod_peps)
    
    return result_peps

def load_peps(pep_dir):
    fasta_files = verify_in_dir(pep_dir, "fasta")
    
    use_mods = config.get_config(key="use_mods", section="input")
    mods_list = config.Mods
    num_mods = config.get_config(key="num_mods", section="search")
    pep_seq_len = config.get_config(key="pep_seq_len", section="ml")
    
    pep_set = set()
    pep_list = []
    masses = []
    modifieds = []
    prot_list = []
    
    tot_pep_count = 0
    for fasta_file in fasta_files:
        print('Reading: {}'.format(fasta_file))
        
        f = open(fasta_file, "r")
        lines = f.readlines()
        f.close()
        
        print("File length: {}".format(len(lines)))
        peps = []
        temp_prot = ""
        with progressbar.ProgressBar(max_value=len(lines)) as bar:
            for i, line in enumerate(lines):
                #line = line.strip().replace("C", "Cc")
                if line.startswith(">"):
                    temp_prot = line[1:].strip()
                    continue
                elif any(x in line for x in config.Ignore):
                    continue
                peps = add_mods(line, mods_list, num_mods) if use_mods else [line]
                for pep in peps:
                    pep = pep.strip()
                    mass = sim.get_pep_mass(pep)
                    modified = any(aa.islower() for aa in pep)
                    if pep not in pep_set and len(pep) <= pep_seq_len:
                        pep_set.add(pep)
                        pep_list.append(pep)
                        masses.append(mass)
                        modifieds.append(modified)
                        prot_list.append(temp_prot)
                        tot_pep_count += 1
                bar.update(i)
        print("Peptides written: {}".format(tot_pep_count))
        
        return pep_list, prot_list, masses, modifieds

In [103]:
def load_specs(spec_dir):
    spec_size = config.get_config(key="spec_size", section="input")
    spec_files = verify_in_dir(spec_dir, "npy")
    spec_ids = []
    spec_list = []
    masses = []
    charges = []
    count = 0
    with progressbar.ProgressBar(max_value=len(spec_files)) as bar:
        for i, spec_file in enumerate(spec_files):
            file_name = spec_file.split('/')[-1]
            file_parts = re.search(r"(\d+)-(\d+.\d+)-(\d+).[pt|npy]", file_name)
            spec_id = int(file_parts[1])
            mass = round(float(file_parts[2]), 2)
            charge = int(file_parts[3])
            if charge > 5:
                continue
            spec_ids.append(spec_id)
            np_spec = np.load(spec_file)
            spec_list.append(np_spec)
            masses.append(mass)
            charges.append(charge)

            count += 1
            bar.update(i)
    print("count: {}".format(count))
    return spec_ids, spec_list, masses, charges

In [141]:
from torch.utils import data
class SpectralDataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, dir_path):
        'Initialization'
        
        in_path = Path(dir_path)
        assert in_path.exists()
        assert in_path.is_dir()
        
        self.spec_path = join(dir_path, "spectra")
        self.spec_size = config.get_config(section='input', key='spec_size')
        
        self.means = torch.from_numpy(np.load(join(dir_path, "means.npy"))).float()
        self.stds  = torch.from_numpy(np.load(join(dir_path, "stds.npy"))).float()
        
        spec_ids, spec_lst, spec_mass_lst, spec_charge_lst = load_specs(self.spec_path)
        all_sorts = list(zip(*sorted(zip(spec_ids, spec_lst, spec_mass_lst, spec_charge_lst), key=lambda x: x[2])))
        self.spec_ids         = all_sorts[0]
        self.spec_list        = all_sorts[1]
        self.spec_mass_list   = all_sorts[2]
        self.spec_charge_list = all_sorts[3]
        print('Spectral Dataset Size: {}'.format(len(self.spec_list)))
        

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.spec_list)


    def __getitem__(self, index):
        'Generates one sample of data'

        # Load spectra
        np_spec = self.spec_list[index]
        ind = torch.LongTensor([[0]*np_spec.shape[1], np_spec[0]])
        val = torch.FloatTensor(np_spec[1])
        torch_spec = torch.sparse_coo_tensor(
            ind, val, torch.Size([1, self.spec_size])).to_dense()
        torch_spec = (torch_spec - self.means) / self.stds

        return torch_spec


class PeptideDataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, dir_path, decoy=False):
        'Initialization'
        
        in_path = Path(dir_path)
        assert in_path.exists()
        assert in_path.is_dir()

        self.aas            = ['_PAD'] + list(config.AAMass.keys())# + list(config.ModCHAR.values())
        self.aa2idx         = {a:i for i, a in enumerate(self.aas)}
        self.idx2aa         = {i:a for i, a in enumerate(self.aas)}
        
        self.pep_path       = dir_path
        self.vocab_size     = len(self.aa2idx) # + self.charge + self.num_species + 1
        print("Vocabulary Size: {}".format(self.vocab_size))
        self.seq_len        = config.get_config(section='ml', key='pep_seq_len')
        
        pep_lst, prot_list, pep_mass_lst, pep_modified_lst = load_peps(self.pep_path)
        
        self.pep_lst_set = set(pep_lst)
#         out_dir = "/disk/raptor-2/mtari008/data/deepsnap/preprocessed-human-hcd-tryp-best/pts/"
#         with open(join(out_dir, 'pep_pickle.pkl'), 'rb') as f:
#             search_peps = pickle.load(f)
#         added_counter = 0
#         for s_pep in search_peps:
#             if s_pep not in self.pep_lst_set:
#                 self.pep_lst_set.add(s_pep)
#                 added_counter += 1
#                 pep_lst.append(s_pep)
#                 prot_list.append("unknown")
#                 pep_mass_lst.append(sim.get_pep_mass(s_pep))
#                 pep_modified_lst.append(any(aa.islower() for aa in s_pep))
#         print("New peptides added: {}".format(added_counter))
            
            
        all_sorts = list(zip(*sorted(zip(pep_lst, prot_list, pep_mass_lst, pep_modified_lst), key=lambda x: x[2])))
        self.pep_list          = all_sorts[0]
        self.prot_list         = all_sorts[1]
        self.pep_mass_list     = all_sorts[2]
        self.pep_modified_list = all_sorts[3]
        if decoy:
            self.pep_list, self.prot_list, self.pep_mass_list, self.pep_modified_list = self.get_docoys()
            
        print('Peptide Dataset Size: {}'.format(len(self.pep_list)))
        

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.pep_list)


    def __getitem__(self, index):
        'Generates one sample of data'
        pep = self.pep_list[index].strip()
        pepl = [self.aa2idx[aa] for aa in pep]
        pepl = self.pad_left(pepl)
        torch_pep = torch.tensor(pepl, dtype=torch.long)
        return torch_pep
        

    def pad_left(self, arr):
        out = np.zeros(self.seq_len)
        out[-len(arr):] = arr
        return out
    
    def get_docoys(self):
        decoy_list = []
        decoy_prot_list = []
        decoy_mass_list = []
        decoy_modified_list = []
        for pep, prot, mass, modified in zip(self.pep_list, self.prot_list, self.pep_mass_list, self.pep_modified_list):
            pep_parts = re.findall(r"([A-Z][a-z]?)", pep)
            decoy_pep = pep_parts[0] + "".join(pep_parts[-2:0:-1]) + pep_parts[-1]
            if decoy_pep not in self.pep_lst_set:
                decoy_list.append(decoy_pep)
                decoy_prot_list.append(prot)
                decoy_mass_list.append(mass)
                decoy_modified_list.append(modified)
        return decoy_list, decoy_prot_list, decoy_mass_list, decoy_modified_list

In [153]:
spec_dir = "/disk/raptor-2/mtari008/data/deepsnap/preprocessed"
#spec_dir = "/disk/raptor-2/mtari008/data/deepsnap/preprocessed-human-hcd-tryp-best/pts"
pep_dir = "/disk/raptor-2/mtari008/data/deepsnap/peps"

spec_dataset = SpectralDataset(spec_dir)
pep_dataset = PeptideDataset(pep_dir)
dec_dataset = PeptideDataset(pep_dir, decoy=True)

100% (1036792 of 1036792) |##############| Elapsed Time: 0:07:21 Time:  0:07:21


count: 1033710
Spectral Dataset Size: 1033710
Vocabulary Size: 30
Reading: /disk/raptor-2/mtari008/data/deepsnap/peps/human-peptidome.fasta


  0% (28029 of 11384076) |               | Elapsed Time: 0:00:00 ETA:   0:01:21

File length: 11384076


100% (11384076 of 11384076) |############| Elapsed Time: 0:02:02 Time:  0:02:02


Peptides written: 3893139
Peptide Dataset Size: 3893139
Vocabulary Size: 30
Reading: /disk/raptor-2/mtari008/data/deepsnap/peps/human-peptidome.fasta


  0% (27253 of 11384076) |               | Elapsed Time: 0:00:00 ETA:   0:01:23

File length: 11384076


100% (11384076 of 11384076) |############| Elapsed Time: 0:01:48 Time:  0:01:48


Peptides written: 3893139
Peptide Dataset Size: 3890768


In [143]:
def spec_collate(batch):
    specs = torch.stack([item for item in batch], 0)
    dummy_pep = np.zeros(config.get_config(section="ml", key="pep_seq_len"))
    dummy_pep = torch.from_numpy(dummy_pep).long().unsqueeze(0)
    return [specs, dummy_pep]

def pep_collate(batch):
    peps = torch.stack([item for item in batch], 0)
    dummy_spec = np.zeros(config.get_config(section="input", key="spec_size"))
    dummy_spec = torch.from_numpy(dummy_spec).float().unsqueeze(0)
    return [dummy_spec, peps]

spec_batch_size = 16384
pep_batch_size = 4096
spec_loader = torch.utils.data.DataLoader(
        dataset=spec_dataset, batch_size=spec_batch_size,
        collate_fn=spec_collate)
pep_loader = torch.utils.data.DataLoader(
        dataset=pep_dataset, batch_size=pep_batch_size,
        collate_fn=pep_collate)
dec_loader = torch.utils.data.DataLoader(
        dataset=dec_dataset, batch_size=pep_batch_size,
        collate_fn=pep_collate)

In [144]:
def runModel(loader, s_model, in_type):
    with torch.no_grad():
        accurate_labels = 0
        all_labels = 0
        loss = 0
        h = s_model.module.init_hidden(loader.batch_size)
        out_out = torch.Tensor().cpu()
        with progressbar.ProgressBar(max_value=len(loader)) as bar:
            for batch_idx, batch in enumerate(loader):
                batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
                h = tuple([e.data[:, :len(batch[1]), :].contiguous() for e in h])
                out, h = s_model(batch, h, data_type=in_type)
                out_out = torch.cat((out_out, out.to("cpu")), dim=0)
                bar.update(batch_idx)
        del h
        return out_out

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12349'
dist.init_process_group(backend='nccl', world_size=1, rank=0)
snap_model = torch.load('models/hcd/model-108-91.27-NoHCDTrypBest.pt').to(device)
snap_model.eval()
snap_model

cuda


DistributedDataParallel(
  (module): Net(
    (embedding): Embedding(30, 256)
    (lstm): LSTM(256, 512, batch_first=True, bidirectional=True)
    (linear1_1): Linear(in_features=80000, out_features=512, bias=True)
    (linear1_2): Linear(in_features=512, out_features=256, bias=True)
    (linear2_1): Linear(in_features=1024, out_features=512, bias=True)
    (linear2_2): Linear(in_features=512, out_features=256, bias=True)
    (dropout1): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
  )
)

In [16]:
torch.cuda.empty_cache()
for obj in gc.get_objects():
    try:
        if (torch.is_tensor(obj) and obj.is_cude) or (hasattr(obj, 'data') and torch.is_tensor(obj.data) and obj.data.is_cuda):
            print(type(obj), obj.size())
    except:
        pass

/lclhome/mtari008/anaconda3/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:102: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


In [19]:
torch.cuda.empty_cache()

In [20]:
gc.collect()

62

In [18]:
import psutil
def memReport():
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            print(type(obj), obj.size())
            del obj
    
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()
memReport()

3.7.7 (default, Mar 26 2020, 15:48:22) 
[GCC 7.3.0]
1.9
svmem(total=134824566784, available=116430745600, percent=13.6, used=17510600704, free=42559819776, active=47931006976, inactive=17815711744, buffers=18163945472, cached=56590200832, shared=70180864, slab=24808374272)
memory GB: 13.952167510986328
<class 'torch.Tensor'> torch.Size([80000])
<class 'torch.Tensor'> torch.Size([80000])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([256, 512])
<class 'torch.nn.parameter.Parameter'> torch.Size([512])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 1024])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([256, 512])
<class 'torch.nn.parameter.Parameter'> torch.Size([512])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 80000])
<class 'torch.nn.parameter.Parameter'> torch.Size([2048])
<class 'torch.nn.parameter.Parameter'> torch.Size([2048])
<class 'torch.n

In [145]:
torch.cuda.empty_cache()
e_specs = runModel(spec_loader, snap_model, "specs")
torch.cuda.empty_cache()
gc.collect()
print("Spectra done!")

100% (3 of 3) |##########################| Elapsed Time: 0:00:09 Time:  0:00:09


Spectra done!


In [146]:
e_peps = runModel(pep_loader, snap_model, "peps")
torch.cuda.empty_cache()
gc.collect()
print("Peptides done!")
e_decs = runModel(dec_loader, snap_model, "peps")
torch.cuda.empty_cache()
gc.collect()
print("Decoys done!")

100% (951 of 951) |######################| Elapsed Time: 0:10:00 Time:  0:10:00


Peptides done!


100% (950 of 950) |######################| Elapsed Time: 0:07:20 Time:  0:07:20


Decoys done!


In [29]:
torch.cuda.empty_cache()

In [83]:
print(e_specs.shape, e_peps.shape, e_decs.shape)
torch.save(e_specs, "/lclhome/mtari008/mtari008/data/deepsnap/embeds/e-specs.pt")
torch.save(e_peps,  "/lclhome/mtari008/mtari008/data/deepsnap/embeds/e-peps.pt")
torch.save(e_decs,  "/lclhome/mtari008/mtari008/data/deepsnap/embeds/e-decs.pt")

torch.Size([40339, 256]) torch.Size([6520722, 256]) torch.Size([6516239, 256])


In [31]:
e_specs = torch.load("/lclhome/mtari008/mtari008/data/deepsnap/embeds/e-specs.pt")
e_peps  = torch.load("/lclhome/mtari008/mtari008/data/deepsnap/embeds/e-peps.pt")
e_decs  = torch.load("/lclhome/mtari008/mtari008/data/deepsnap/embeds/e-decs.pt")

In [147]:
def get_search_mask(spec_masses, pep_masses, tol):
    l_tol = tol + 0.01
    rows = []
    cols = []
    pep_min = pep_max = 0
    for row_id, spec_mass in enumerate(spec_masses):
        min_mass = max(spec_mass - l_tol, 0.0)
        max_mass = spec_mass + l_tol
        while (pep_min < len(pep_masses) and 
               min_mass > pep_masses[pep_min]):
            pep_min += 1
        while (pep_max < len(pep_masses) and 
               max_mass > pep_masses[pep_max]):
            pep_max += 1
            
        rows.extend([row_id] * (pep_max - pep_min))
        cols.extend(range(pep_min, pep_max))
    
    assert len(rows) == len(cols)
    mask = torch.zeros(len(spec_masses), len(pep_masses))
    mask[rows, cols] = 1
    return mask

In [148]:
def search(search_loader, tol):
    pep_sort_inds = []
    pep_sort_vals = []
    dec_sort_inds = []
    dec_sort_vals = []
    with progressbar.ProgressBar(max_value=len(search_loader)) as bar:
        for idx, spec_batch in enumerate(search_loader):
            l_tol = tol + 0.01
            batch_size = search_loader.batch_size
            st = idx * batch_size
            en = st + batch_size
            spec_masses = spec_dataset.spec_mass_list[st:en]
            min_mass = max(spec_masses[0] - l_tol, 0)
            max_mass = spec_masses[-1] + l_tol
            pep_min = pep_max = 0

            while (pep_min < len(pep_dataset.pep_mass_list) and 
                   min_mass - pep_dataset.pep_mass_list[pep_min] > 0.01):
                pep_min += 1
            while (pep_max < len(pep_dataset.pep_mass_list) and
                   max_mass - pep_dataset.pep_mass_list[pep_max] >= 0.01):
                pep_max += 1

            dec_min = dec_max = 0
            while (dec_min < len(dec_dataset.pep_mass_list) and
                   min_mass - dec_dataset.pep_mass_list[dec_min] > 0.01):
                dec_min += 1
            while (dec_max < len(dec_dataset.pep_mass_list) and 
                   max_mass - dec_dataset.pep_mass_list[dec_max] >= 0.01):
                dec_max += 1

            pep_batch = e_peps[pep_min:pep_max]
            pep_masses = pep_dataset.pep_mass_list[pep_min:pep_max]
            dec_batch = e_decs[dec_min:dec_max]
            dec_masses = dec_dataset.pep_mass_list[dec_min:dec_max]

            spec_batch = spec_batch.to(device)
            #print("pep batch len: {}".format(len(pep_batch)))
            l_pep_batch_size = 16384
            pep_loader = torch.utils.data.DataLoader(
                dataset=pep_batch, batch_size=l_pep_batch_size)
            l_pep_dist = []
            for pep_idx, l_pep_batch in enumerate(pep_loader):
                l_pep_batch = l_pep_batch.to(device)
                l_st = pep_idx * l_pep_batch_size
                l_en = l_st + l_pep_batch_size
                l_pep_masses = pep_masses[l_st:l_en]
                spec_pep_mask = get_search_mask(spec_masses, l_pep_masses, tol).to(device)
                # spec_pep_mask[spec_pep_mask == 0] = float("inf")
                spec_pep_dist = 1.0 / process.pairwise_distances(spec_batch, l_pep_batch)
                l_pep_dist.append((spec_pep_dist * spec_pep_mask).to("cpu"))
            
            l_pep_dist.append(torch.zeros(len(spec_batch), keep_psms + 1))
            pep_sort = torch.cat(l_pep_dist, 1)
            pep_lcn = np.ma.masked_array(pep_sort, mask=pep_sort==0).min(1).data
            pep_sort = pep_sort.sort(descending=True)
            pep_sort_inds.append(pep_sort.indices[:, :keep_psms + 1] + pep_min) # offset for the global array
            pep_sort_vals.append(torch.cat((pep_sort.values[:, :keep_psms + 1], 
                                            torch.from_numpy(pep_lcn).unsqueeze(1)), 1))
            
            dec_loader = torch.utils.data.DataLoader(
                dataset=dec_batch, batch_size=l_pep_batch_size)
            l_dec_dist = []
            for dec_idx, l_dec_batch in enumerate(dec_loader):
                l_dec_batch = l_dec_batch.to(device)
                l_st = dec_idx * l_pep_batch_size
                l_en = l_st + l_pep_batch_size
                l_dec_masses = dec_masses[l_st:l_en]
                spec_dec_mask = get_search_mask(spec_masses, l_dec_masses, tol).to(device)
                # spec_dec_mask[spec_dec_mask == 0] = float("inf")
                spec_dec_dist = 1.0 / process.pairwise_distances(spec_batch, l_dec_batch)
                l_dec_dist.append((spec_dec_dist * spec_dec_mask).to("cpu"))
            
            l_dec_dist.append(torch.zeros(len(spec_batch), keep_psms + 1))
            dec_sort = torch.cat(l_dec_dist, 1)
            dec_lcn = np.ma.masked_array(dec_sort, mask=dec_sort==0).min(1).data
            dec_sort = dec_sort.sort(descending=True)
            dec_sort_inds.append(dec_sort.indices[:, :keep_psms + 1] + dec_min) # offset for the global array
            dec_sort_vals.append(torch.cat((dec_sort.values[:, :keep_psms + 1], 
                                            torch.from_numpy(dec_lcn).unsqueeze(1)), 1))
            
            bar.update(idx)
    
    pep_inds = torch.cat(pep_sort_inds, 0)
    pep_vals = torch.cat(pep_sort_vals, 0)
    dec_inds = torch.cat(dec_sort_inds, 0)
    dec_vals = torch.cat(dec_sort_vals, 0)
    return pep_inds, pep_vals, dec_inds, dec_vals

In [149]:
search_spec_batch_size = 1024
precursor_tolerance = 1.0
keep_psms = 5
search_loader = torch.utils.data.DataLoader(
    dataset=e_specs, batch_size=search_spec_batch_size, shuffle=False)
pep_inds, pep_vals, dec_inds, dec_vals = search(search_loader, precursor_tolerance)

100% (40 of 40) |########################| Elapsed Time: 0:08:03 Time:  0:08:03


In [35]:
torch.save(pep_inds, "/lclhome/mtari008/mtari008/data/deepsnap/results-human-hcd-tryp-best/pep-inds.pt")
torch.save(pep_vals, "/lclhome/mtari008/mtari008/data/deepsnap/results-human-hcd-tryp-best/pep-vals.pt")
torch.save(dec_inds, "/lclhome/mtari008/mtari008/data/deepsnap/results-human-hcd-tryp-best/dec-inds.pt")
torch.save(dec_vals, "/lclhome/mtari008/mtari008/data/deepsnap/results-human-hcd-tryp-best/dec-vals.pt")

In [15]:
pep_inds = torch.load("/lclhome/mtari008/mtari008/data/deepsnap/results/pep-inds.pt")
pep_vals = torch.load("/lclhome/mtari008/mtari008/data/deepsnap/results/pep-vals.pt")
dec_inds = torch.load("/lclhome/mtari008/mtari008/data/deepsnap/results/dec-inds.pt")
dec_vals = torch.load("/lclhome/mtari008/mtari008/data/deepsnap/results/dec-vals.pt")

In [150]:
def generate_percolator_input(l_pep_inds, l_pep_vals, pd_dataset, res_type):
    assert res_type == "target" or res_type == "decoy"
    assert len(l_pep_inds) == len(l_pep_vals)
    pin_charge = config.get_config(section="search", key="charge")
    l_global_out = []
    tot_count = 0
    for idx, (pep_inds_row, pep_vals_row) in enumerate(zip(l_pep_inds, l_pep_vals)):
        # Reminder: pep_inds_row length is one less than pep_vals_row
        for iidx in range(len(pep_inds_row) - 1):
            pep_ind = pep_inds_row[iidx]
            pep_val = pep_vals_row[iidx]
            if pep_val.item() > 0:
                charge = [0] * pin_charge
                charge[spec_dataset.spec_charge_list[idx] - 1] = 1
                label = 1 if res_type == "target" else -1
                out_row = [f"{res_type}-{tot_count}", label, idx, pep_val.item()]
                out_row.append(spec_dataset.spec_mass_list[idx])
                out_row.append(pd_dataset.pep_mass_list[pep_ind.item()])
                out_row.append((pep_val - pep_vals_row[iidx + 1]).item())
                out_row.append((pep_val - pep_vals_row[-1]).item())
                out_row.extend(charge)
                out_pep = pd_dataset.pep_list[pep_ind.item()]
                out_pep_array = []
                for aa in out_pep:
                    if aa.islower():
                        out_pep_array.append("["+str(config.AAMass[aa])+"]")
                    else:
                        out_pep_array.append(aa)
                out_pep = "".join(out_pep_array)
                out_prot = pd_dataset.prot_list[pep_ind.item()]
                pep_len = sum([a.isupper() for a in out_pep])
                out_row.append(pep_len)
                out_row.append(out_pep)
                out_row.append(out_prot)
                l_global_out.append(out_row)
                tot_count += 1
    return l_global_out

In [151]:
global_out = generate_percolator_input(pep_inds, pep_vals, pep_dataset, "target")
global_out.extend(generate_percolator_input(dec_inds, dec_vals, dec_dataset, "decoy"))

In [152]:
pin_charge = config.get_config(section="search", key="charge")
charge_cols = [f"charge-{ch+1}" for ch in range(pin_charge)]
cols = ["SpecId", "Label", "ScanNr", "SNAP", "ExpMass", "CalcMass", "deltCn", "deltLCn"] + charge_cols + ["PepLen", "Peptide", "Proteins"]
df = pd.DataFrame(global_out, columns=cols)
df.sort_values(by="SNAP", inplace=True, ascending=False)
df.to_csv("/lclhome/mtari008/mtari008/data/deepsnap/results-human-hcd-tryp-best/res-pin.tab", sep="\t", index=False)

In [3]:
sort_col = [x[0] for x in np.array(pep_vals)]
sort_col[-10:]

[0.71178627,
 0.82731843,
 0.8814876,
 0.9404478,
 0.90577126,
 0.7447554,
 0.87054515,
 0.81516623,
 1.0211788,
 0.84863305]

In [27]:
df = pd.DataFrame(list(zip(np.array(pep_inds), np.array(pep_vals), 
                           np.array(dec_inds), np.array(dec_vals), np.array(sort_col))), 
                  columns=["pep_inds", "pep_vals", "dec_inds", "dec_vals", "sort_col"])

df.sort_values(by="sort_col", inplace=True, ascending=False)

NameError: name 'sort_col' is not defined

In [6]:
df.to_csv("/lclhome/mtari008/mtari008/data/deepsnap/results/output.csv")

In [5]:
pd.__version__

'1.0.5'

In [ ]:
def fastaToSpectraBatch(filePath, spectra_batch_size):    
    f = open(filePath)
    lines = f.readlines()
    f.close()
    masses = []
    spectra_out = torch.Tensor().to(device)
    peps = []
    
    dh = display('0%', display_id=True)
    
    start = 0
    i = 0
    while start < len(lines):
        print('Batch: ' + str(i))
        i += 1
        
        print('Generating spectra...')
        spectra, l_masses, l_peps = fastatospectra(lines, start, spectra_batch_size, dh)
        masses.extend(l_masses)
        peps.extend(l_peps)
        start = start + spectra_batch_size
        
        with torch.no_grad():
            print('Converting to tensor...')
            '''dtype=torch.float'''
            spectra = np.asarray(spectra)
            spectraTensor = torch.as_tensor(spectra, dtype=torch.float)[:, None, :]
            spectra_loader = torch.utils.data.DataLoader(dataset=spectraTensor, batch_size=batch_size, shuffle=False)
            
            print('Running the model...')
            model_out = runModel(spectra_loader)
            spectra_out = torch.cat((spectra_out, model_out), dim=0)
            
    return spectra_out, masses, peps

In [ ]:
spectra_batch_size = 500000
#targetPath = '/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.target.txt'
#decoyPath = '/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.decoy.txt'
targetPath = '/disk/raptor/lclhome/mtari008/Proteomics/crux/crux-output/generate-peptides.target.txt'
decoyPath = '/disk/raptor/lclhome/mtari008/Proteomics/crux/crux-output/generate-peptides.decoy.txt'
targetspectra_out, targetmasses, targetpeptides = fastaToSpectraBatch(targetPath, spectra_batch_size)
decoyspectra_out, decoymasses, decoypeptides = fastaToSpectraBatch(decoyPath, spectra_batch_size)


print(len(targetspectra_out), len(targetmasses))
print(len(decoyspectra_out), len(decoymasses))

In [ ]:
nearest_target_distances, nearest_target_indexes = process.pairwise_distances(
    queryspectra_out, targetspectra_out).min(1)

nearest_decoy_distances, nearest_decoy_indexes  = process.pairwise_distances(
    queryspectra_out,  decoyspectra_out).min(1)

nearest_distances, target_decoy = torch.cat((nearest_target_distances.view(1, -1), 
                                             nearest_decoy_distances.view(1, -1)), dim=0).min(0)

print(target_decoy.to('cpu'))

sorted_nearest_distances, snd_index = nearest_distances.sort()
target_decoy = target_decoy[snd_index]

print(target_decoy)

## To lookup a peptide corresponding to a value in sorted_nearest_distances at index i:
## td = target_deocy[i]            # a binary array. 0: target, 1: decoy
## query_index = snd_index[i]        # lookup the index of query spectrum
## pep_index = nearest_decoy_indexes[query_index] if td else nearest_target_indexes[query_index]
## td_peptide = decoy_peptides[pep_index] if td else target_peptides[pep_index]

In [ ]:
def fdr(td, target_fdr):
    target_count = 0
    decoy_count = 0
    q_values = [0] * len(td)
    for idx, val in enumerate(td):
        if val == 0:
            target_count += 1
        else:
            decoy_count += 1
        return_fdr = decoy_count / (idx + 1)
        print('{} / {} = {}'.format(decoy_count, idx+1, decoy_count/(idx+1)))
        #if return_fdr > target_fdr:
        #    return target_count, return_fdr
    return target_count, return_fdr

In [ ]:
print(fdr(target_decoy, 0.01))

In [ ]:
print(targetmasses[0:10])
print(len(decoypeptides))
targetspectra_out, targetmasses, targetpeptides = list(zip(*sorted(zip(targetspectra_out, targetmasses, targetpeptides), 
                                                                   key=lambda pair: pair[1])))
decoyspectra_out, decoymasses, decoypeptides = list(zip(*sorted(zip(decoyspectra_out, decoymasses, decoypeptides),
                                                                key=lambda pair: pair[1])))

In [ ]:
print(len(targetmasses))
print(targetmasses[1000:1010])
print(decoymasses[1000:1010])

In [ ]:
def find_range(arr, val, tolerance=3):
    assert len(arr) > 0
    assert tolerance > 0
    assert val > 0
    
    left = val - tolerance if val - tolerance >= 0 else 0
    right = val + tolerance
    
    ileft = bisect.bisect_left(arr, left)
    iright = bisect.bisect_right(arr, right)
    
    return (ileft, iright-1) if iright - ileft > 0 else (-1, -1)
    
# Function to insert element 
def insert(lst, n): 
    assert not lst or len(lst[0]) == len(n)
    
    index = num_psm
    # Searching for the position 
    for i in range(len(lst)): 
        if lst[i][0] > n[0]: 
            index = i 
            break
      
    # Inserting n in the list 
    lst = lst[:index] + [n] + lst[index:] 
    return lst[:num_psm]

In [ ]:
def calculate_L2_Dist(queryspectra, querymasses, dbspectra, dbmasses, pre_tol=3, keep_psm=1):
    prev = 0
    psm_scores = []
    psm_pepids = []
    
    for ind, (spec, spec_mass) in enumerate(zip(queryspectra, querymasses)):
        left, right = find_range(dbmasses, spec_mass, pre_tol)
        if left != -1 and right != -1:
            candidate_specs = dbspectra[left:right+1]
            l_scores = np.linalg.norm(np.subtract(candidate_specs, spec), axis=1)**2
            psm_pepids.append(left + l_scores.argsort()[:keep_psm]) #add argsort later
            psm_scores.append(np.sort(l_scores)[:keep_psm])
            #print(ind)
            new = int((ind/len(queryspectra)) * 100)
            if new > prev:
                clear_output(wait=True)
                print(str(new) + '%')
                prev = new
        else:
            print('***********error**************')
            print('ind: ' + str(ind))
            print('spec_mass: ' + str(spec_mass))
            print('**********end error************')
            
    return psm_scores, psm_pepids

In [ ]:
def calculate_xcorr(queryspectra, querymasses, dbspectra, dbmasses, pre_tol=3, keep_psm=1):
    prev = 0
    psm_scores = []
    psm_pepids = []
    
    for ind, (spec, spec_mass) in enumerate(zip(queryspectra, querymasses)):
        left, right = find_range(dbmasses, spec_mass, pre_tol)
        if left != -1 and right != -1:
            candidate_specs = dbspectra[left:right+1]
            l_scores = np.dot(candidate_specs, spec)
            psm_pepids.append(left + l_scores.argsort()[::-1][:keep_psm]) #add argsort later
            psm_scores.append(np.sort(l_scores)[::-1][:keep_psm])
            #print(ind)
            new = int((ind/len(queryspectra)) * 100)
            if new > prev:
                clear_output(wait=True)
                print(str(new) + '%')
                prev = new
        else:
            print('***********error**************')
            print('ind: ' + str(ind))
            print('spec_mass: ' + str(spec_mass))
            print('**********end error************')
            
    return psm_scores, psm_pepids

In [ ]:
# targetpsm_scores, targetpsm_ids = calculate_L2_Dist(queryspectra_out, spectramasses, targetspectra_out,
#                                      targetmasses, precursor_tolerance, num_psm)
# decoypsm_scores, decoypsm_ids = calculate_L2_Dist(queryspectra_out, spectramasses, decoyspectra_out,
#                                      decoymasses, precursor_tolerance, num_psm)
targetpsm_scores, targetpsm_ids = calculate_xcorr(queryspectra_out, spectramasses, targetspectra_out,
                                     targetmasses, precursor_tolerance, num_psm)
decoypsm_scores, decoypsm_ids = calculate_xcorr(queryspectra_out, spectramasses, decoyspectra_out,
                                     decoymasses, precursor_tolerance, num_psm)

In [ ]:
print(targetmasses[2000000:2000010])

In [ ]:
print(len(targetspectra_out))
print(len(decoyspectra_out))

print(len(targetpsm_scores))
print(len(decoypsm_scores))

print(targetpsm_scores[0:10])
print(decoypsm_scores[0:10])

targetpsm_scores = np.asarray(targetpsm_scores).flatten()
decoypsm_scores = np.asarray(decoypsm_scores).flatten()

targetpsm_ids = np.asarray(targetpsm_ids).flatten()
decoypsm_ids = np.asarray(decoypsm_ids).flatten()

targetpsm_peps = np.asarray(targetpeptides)[targetpsm_ids]
decoypsm_peps = np.asarray(decoypeptides)[decoypsm_ids]

#targetpsm_ids = targetpsm_ids[np.argsort(np.asarray(targetpsm_scores).flatten())]
#decoypsm_ids = decoypsm_ids[np.argsort(np.asarray(decoypsm_scores).flatten())]

sorted_targets = np.sort(np.asarray(targetpsm_scores).flatten())[::-1]
sorted_decoys = np.sort(np.asarray(decoypsm_scores).flatten())[::-1]

print(targetpsm_ids[0:10])
print(decoypsm_ids[0:10])

print(sorted_targets[0:100])
print(sorted_decoys[0:100])

#targetpsm_scores = np.column_stack((targetpsm_scores, np.ones(len(targetpsm_scores))))
#decoypsm_scores = np.column_stack((decoypsm_scores, np.zeros(len(decoypsm_scores))))
#decoypsm_scores = np.asarray(decoypsm_scores)

In [ ]:
targetpsm_scores

In [ ]:
df = pd.DataFrame(
    data={'spectrum_ids':np.arange(len(targetpsm_peps)), 'target_psms':targetpsm_peps, 
          'targetpsm_scores':targetpsm_scores, 'decoy_psms':decoypsm_peps, 'decoypsm_scores':decoypsm_scores})
df.to_csv('deepsearch.csv')

In [ ]:
sortedpeps = np.array(sortedpeps)
correct_matches = 0
for ind, psms in enumerate(psm_scores):
    #print('ind: ' + str(ind) + ', mass: ' + str(sortedspecmasses[ind]))
    #print('psms: ' + str(psms) + ', psm mass: ' + str(sortedpepmasses[psms[0]]))
    cand_peps = sortedpeps[psms]
    #print(cand_peps)
    for pep in cand_peps:
        #print('pepidmass: ' + str(pepidmass[pep][0]))
        #print('ids: ' + str(ids[ind]) + ', in: ' + str(pepidmass[pep][0]))
        if ids[ind] in pepidmass[pep][0]:
            correct_matches += 1
            break
print(len(psm_scores))
print(correct_matches)
print(str((correct_matches/len(psm_scores)) * 100) + '% correct matches in top ' + str(num_psm) + ' psms.')

# Scratch Work:

In [21]:
pin_charge = config.get_config(section="search", key="charge")
charge_cols = [f"charge-{ch+1}" for ch in range(pin_charge)]
print(charge_cols)
a = ["a", "b", "c"]
b = a + charge_cols
print(b)

['charge-1', 'charge-2', 'charge-3', 'charge-4', 'charge-5']
['a', 'b', 'c', 'charge-1', 'charge-2', 'charge-3', 'charge-4', 'charge-5']


In [ ]:
file = open('/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.target.txt')
tlines = file.readlines()
print(len(tlines))
tpeps = list((tline.split('\t')[0], float(tline.split('\t')[1])) for tline in tlines)
tpeps = sorted(tpeps, key=lambda pair: pair[1])
print(tpeps[10000:10010])
file.close()

In [ ]:
file = open('/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.decoy.txt')
tlines = file.readlines()
print(len(tlines))
tpeps = list((tline.split('\t')[0], float(tline.split('\t')[1])) for tline in tlines)
tpeps = sorted(tpeps, key=lambda pair: pair[1])
print(tpeps[10000:10010])
file.close()

In [ ]:
np.arange(150)

In [ ]:
x = torch.ones(2, 3)
z = (torch.ones(3) * 5).view(1, 3)
print(x)
print(z)
# or this
#z = torch.ones(2, 3, 1) * 5
y = torch.cat((x, z), dim=0)
print(y)

In [ ]:
## Keep this cell....
## How to sort a 2D tensor by its first row.....
x = torch.Tensor([5., 3., 4., 1., 2.]).view(1, -1)
z = torch.Tensor([1., 3., 2., 5., 4.]).view(1, -1)
print(x)
print(z)
# or this
#z = torch.ones(2, 3, 1) * 5
y = torch.cat((x, z), dim=0)
_, sorted_index_y = y[0, :].sort()
sorted_y = y[:, sorted_index_y]
print(y)
print(sorted_y)

In [ ]:
regex = r"Name\:\s(?P<pep>[a-zA-Z]+)\/(?P<charge>\d+)(?:\_(?P<num_mods>\d+)(?P<mods>.*))?"
test_str = "Name: AAAAAEEGMEPR/2"
m = re.search(regex, test_str)
print((m['pep']))
print(type(int(m.group('charge'))))

print(bool(m['pep']))

In [ ]:
a = float(re.findall(r"MW\:\s([-+]?[0-9]*\.?[0-9]*)", "MW: 1261.5610")[0])
print(type(a))

In [24]:
dic = {'ABC':(123.5, True), 'CD':(544.2, False), 'GHTE':(7654.445, True)}
df = pd.DataFrame.from_dict(dic, orient="index", columns=["mass", "modified"])
df.index.name = "pep"
print(list(df.index))

['ABC', 'CD', 'GHTE']


In [35]:
import re
pep = "ABCiDaE"
pep_parts = re.findall(r"([A-Z][a-z]?)", pep)
pep = pep_parts[0] + "".join(pep_parts[-2:0:-1]) + pep_parts[-1]
print(pep)

ADaCiBE


In [62]:
def pairwise_distances(x, y=None):
    """
    Input: x is a Nxd matrix
           y is an optional Mxd matirx
    Output: dist is a NxM matrix where dist[i,j] is the square norm between x[i,:] and y[j,:]
            if y is not given then use 'y=x'.
    i.e. dist[i,j] = ||x[i,:]-y[j,:]||^2
    """
    x_norm = (x ** 2).sum(1).view(-1, 1)
    print("x norm:")
    print(x_norm)
    if y is not None:
        y_t = torch.transpose(y, 0, 1)
        print("y_t:")
        print(y_t)
        y_norm = (y ** 2).sum(1).view(1, -1)
        print("y_norm:")
        print(y_norm)
    else:
        y_t = torch.transpose(x, 0, 1)
        y_norm = x_norm.view(1, -1)

    dist = x_norm + y_norm - 2.0 * torch.mm(x, y_t)
    # Ensure diagonal is zero if x=y
    if y is None:
        dist = dist - torch.diag(dist.diag())
    dist[dist != dist] = 0  # set all nan values to zero
    return torch.clamp(dist, 0.0, np.inf)

In [67]:
x = torch.tensor([[3., 2., 0., 1., 6.], [8., 4., 5., 1., 6.], [0., 7., 2., 1., 6.]]).tolist()
y = torch.tensor([[0., 2., 0., 1., 1.], [1., 4., 5., 5., 6.], [0., 7., 2., 1., 6.]]).tolist()
x = set(map(tuple, x))
y = set(map(tuple, y))
z = x - x.intersection(y)
print(torch.tensor(list(z), dtype=float))
#print(set.intersection(set(*map(frozenset, x))))
#z = torch.cat((x, y), axis=1)
#print(z)
#print(x)
#print(torch.from_numpy(np.ma.masked_array(x, mask=x==0).min(1).data).unsqueeze(1))
#x[range(3), torch.from_numpy(np.ma.masked_array(x, mask=x==0).min(1))]
# for idx in np.nonzero(x):
#     print(idx)
#     print(x[tuple(idx)])
# y = torch.tensor([[9, 9], [9, 9], [9, 9]])
# z = torch.cat((x, y), 1)
# print(z.shape[1])
# x = torch.tensor([3., 2., 0., 1., 6., 0., 1, 9.])
# x[np.nonzero(x)]

tensor([[3., 2., 0., 1., 6.],
        [8., 4., 5., 1., 6.]], dtype=torch.float64)


In [81]:
m1 = [1.1, 2.2, 3.3, 4.4, 5.5, 6.6]
m2 = [0.1, 1.2, 1.3, 1.4, 2.5, 2.6, 2.7, 3.8, 3.9, 4.1, 4.2, 4.3, 4.4, 5.1, 5.2, 5.3, 5.4, 6.1, 6.3, 6.4, 6.6, 6.7, 6.9]
print(get_search_mask(m1, m2, 1.0))
m3 = torch.tensor(m1) + 9.9
print(m3)

tensor([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
         1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         1., 1., 1., 1., 1.]])
tensor([11.0000, 12.1000, 13.2000, 14.3000, 15.4000, 16.5000])


In [8]:
a = torch.rand(1024, 1582).to("cuda")
b = torch.rand(1024, 1582)
c = a * b.to("cuda")
print(c.shape)

torch.Size([1024, 1582])


In [ ]:
from collections import OrderedDict
od = OrderedDict()